# Empirical Expected Points Model

**Authors:**  
Iain Muir, iam9ez  
Hriday Singh,  
Connor Smith,

*Date: September 9th, 2021*

## Table of Contents

* 0. Import Libraries
* 1. Load Data
* 2. Transform Data
    * 2.1 Remove Garbage Time
    * 2.2 Split Score to Home/Away
    * 2.3 Cast datetime Variables
    * 2.4 Group Point Periods
* 3. Expected Points Model

### 0. Import Libraries

In [2]:
import pandas as pd
import numpy as np

### 1.0 Load Data

In [3]:
PATH = '/Users/iainmuir/Desktop/4Y 1S/STAT 4800/Dataset/2019 PFF All Plays.csv'
DATA = pd.read_csv(PATH)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
DATA.shape

(167027, 177)

In [5]:
DATA.head()

,pff_PLAYID,pff_GAMEID,pff_GAMEDATE,pff_GAMESEASON,pff_WEEK,pff_GSISGAMEKEY,pff_GSISPLAYID,pff_QUARTER,pff_DOWN,pff_CLOCK,...,pff_STSAFETIES,pff_TACKLE,pff_TACKLEASSIST,pff_TEALIGNMENT,pff_TOUCHDOWN,pff_UNBLOCKEDPRESSURE,pff_VISE,pff_WRALIGNMENT,pff_PLAYCLOCK,pff_RUNPASSOPTION
0,3401248,16800,1/13/20,2019,FC,76953,0,1,0,15:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,3401250,16800,1/13/20,2019,FC,76953,0,1,1,15:00,...,NaN,LAST D18,NaN,R,NaN,NaN,NaN,LWR; SLoWR^; SLiWR,3.0,0
2,3401251,16800,1/13/20,2019,FC,76953,0,1,2,14:35,...,NaN,LAST D01,NaN,NaN,NaN,NaN,NaN,LWR^; SRiWR; SRoWR; RWR^,23.0,0
3,3401252,16800,1/13/20,2019,FC,76953,0,1,1,14:19,...,NaN,LAST D06,NaN,R,NaN,NaN,NaN,LWR^; SRWR; RWR^,17.0,0
4,3401253,16800,1/13/20,2019,FC,76953,0,1,1,13:58,...,NaN,LAST D05,NaN,R,NaN,NaN,NaN,SRiWR; SRoWR^; RWR,25.0,1


In [6]:
DATA.describe()

,pff_PLAYID,pff_GAMEID,pff_GAMESEASON,pff_GSISGAMEKEY,pff_GSISPLAYID,pff_QUARTER,pff_DOWN,pff_BLITZDOG,pff_CATCHABLE,pff_DEEPPASS,...,pff_PASSDEPTH,pff_PASSWIDTH,pff_PLAYENDFIELDPOSITION,pff_PUMPFAKE,pff_QBMOVEDOFFSPOT,pff_QBRESET,pff_RUNCONCEPT3,pff_SCOREDIFFERENTIAL,pff_PLAYCLOCK,pff_RUNPASSOPTION
count,1.670270e+05,167027.000000,167027.0,167027.000000,167027.000000,167027.000000,167027.000000,167027.000000,70988.000000,167027.000000,...,67106.000000,58114.000000,167027.000000,167027.000000,70988.000000,70988.000000,0.0,167027.000000,137584.000000,167027.000000
mean,3.187349e+06,15459.272088,2019.0,74972.614553,2011.392799,2.488831,1.789022,0.125944,0.560081,0.058320,...,8.061038,27.194910,1.464524,0.015578,0.205316,0.017989,NaN,-1.210014,12.988996,0.174451
std,1.211107e+05,462.439957,0.0,1267.438124,1182.639051,1.114170,1.131771,0.331787,0.496381,0.234348,...,11.000766,16.630412,30.528227,0.123838,0.403935,0.132912,NaN,16.022196,7.199358,0.379498
min,2.982310e+06,14901.000000,2019.0,73049.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,-21.000000,0.000000,-49.000000,0.000000,0.000000,0.000000,NaN,-79.000000,0.000000,0.000000
25%,3.083844e+06,15135.000000,2019.0,73810.000000,994.000000,2.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,12.000000,-29.000000,0.000000,0.000000,0.000000,NaN,-10.000000,8.000000,0.000000
50%,3.186109e+06,15368.000000,2019.0,74990.000000,1995.000000,2.000000,2.000000,0.000000,1.000000,0.000000,...,5.000000,28.000000,4.000000,0.000000,0.000000,0.000000,NaN,0.000000,12.000000,0.000000
75%,3.286893e+06,15605.000000,2019.0,76239.000000,2991.000000,3.000000,3.000000,0.000000,1.000000,0.000000,...,13.000000,43.000000,27.000000,0.000000,0.000000,0.000000,NaN,7.000000,18.000000,0.000000
max,3.402800e+06,16800.000000,2019.0,76953.000000,5246.000000,5.000000,4.000000,1.000000,1.000000,1.000000,...,59.000000,53.000000,50.000000,1.000000,1.000000,1.000000,NaN,79.000000,40.000000,1.000000


In [7]:
DATA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167027 entries, 0 to 167026
Columns: 177 entries, pff_PLAYID to pff_RUNPASSOPTION
dtypes: float64(29), int64(31), object(117)
memory usage: 225.6+ MB


In [8]:
d = DATA.copy()

### 2.0 Transform Data

#### 2.1 Remove Garbage Time

In [9]:
d = d.loc[d['pff_GARBAGETIME'] == 0]

In [10]:
d = d.loc[(d['pff_QUARTER'] == 1) | (d['pff_QUARTER'] == 3)]

In [11]:
d = d.reset_index(drop=True)

In [12]:
d.shape

(79912, 177)

#### 2.2 Split Score to Home/Away

In [13]:
def split_score(row):
    """
    
    """
    score = row['pff_SCORE']
    
    if '.' in score:
        h, a = score.split('.')
        if len(a) == 1:
            a = int(a)
            a *= 10
    else:
        h = score
        a = 0
    
    return int(h), int(a)

In [14]:
%%time

d['pff_SCORE'] = d['pff_SCORE'].astype(str)
s = pd.DataFrame(d['pff_SCORE'])
scores = s.apply(
    lambda r: split_score(r),
    axis=1,
    result_type='expand'
)

CPU times: user 20.9 s, sys: 331 ms, total: 21.3 s
Wall time: 23.5 s


In [15]:
scores.columns = ['homeScore', 'awayScore']

In [16]:
d['homeScore'], d['awayScore'] = scores['homeScore'], scores['awayScore']

#### 2.3 Cast Datetime Variables

In [17]:
d['pff_GAMEDATE'] = pd.to_datetime(d['pff_GAMEDATE'])

In [18]:
d['pff_CLOCK'] = pd.to_datetime(d['pff_CLOCK']).dt.time

In [19]:
d.shape

(79912, 179)

### 3.0 Expected Points Added Model

In [233]:
points = list()
MAX = max(d['pff_DISTANCE'])
SEED = 42

YTG_DEV = {
    'long': 4,
    'medium': 3,
    'short': 2,
    'inches': 1
}
YTG_MIN = {
    'long': 10,
    'medium': 6,
    'short': 3,
    'inches': 1
}
POSITION_DEV = 5 # yards

In [238]:
def EPA(down, ytg, position, own):
    """
    
    """
    global points
    
    ytg_dev = 4 if ytg in range(10, MAX) else \
              3 if ytg in range(6, 10) else \
              2 if ytg in range(3, 6) else \
              1
                
    SIMULATIONS = 1000
    for i in range(SIMULATIONS):
        point = run_play(down, ytg, position, own)
        points.append(point)
    
    return np.array(points).mean()

In [239]:
def run_play(down, ytg, position, own):
    dist_ = 'long' if ytg in range(10, MAX) else \
            'medium' if ytg in range(6, 10) else \
            'short' if ytg in range(3, 6) else \
            'inches'
    ytg_dev = YTG_DEV[dist_]
    ytg_min = YTG_MIN[dist_]
    
    position *= -1 if own else 1
    fp_right = position - POSITION_DEV
    fp_left = position + POSITION_DEV
    
    # Off the field left
    if fp_left > 0 and fp_right < 0:
        fp_left = -1
    # Redzone -- shrink deviation to 3 yards
    elif 0 < position < 20:
        fp_right += 2
        fp_left -= 2
    
    samples = d.loc[
        (d['pff_DOWN'] == down) &
        (d['pff_DISTANCE'] >= max(ytg_min, ytg - ytg_dev)) &
        (d['pff_DISTANCE'] <= ytg + ytg_dev) &
        (d['pff_FIELDPOSITION'] >= fp_right) &
        (d['pff_FIELDPOSITION'] <= fp_left)
    ]
    if len(samples) == 0:
        return None
    else:
        sample = samples.sample(n=1, random_state=SEED)
        
        # Define conditions for a TD, FG
        if True:
            score = True
            scored = 6
            
            xps = d.loc[
                (d['pff_DOWN'] == 0) &
                (d['pff_DISTANCE'] == 0) &
                (d['pff_FIELDPOSITION'] == 0)
            ]
            xp = xps.sample(n=1, random_state=SEED)
            result = xp['pff_KICKRESULT']
            scored += 1 if 'MADE' in result else 0
        elif True:
            score = True
            scored = 3
        else:
            score = False
    
    if score:
        return scored
    else:
        return run_play(d, y, p)

In [ ]:
down = int(input('Down: ').strip())
ytg = int(input('Yards to Go: ').strip())
position = int(input('Field Position: ').strip())
own = input('Possession (home/away): ').strip()
own = True if own == 'home' else False

In [ ]:
STATE = {
    'down': down,
    'ytg': ytg,
    'position': position,
    'own': own
}

In [ ]:
epa = EPA(**STATE)

In [ ]:
print('''
    Expected Points Model:
    S (Down: {}, YTG: {}, Position: {}) == {} EP
'''.format(
    STATE['down'],
    STATE['ytg'],
    ('Own ' if STATE['own'] else '') + str(STATE['position']),
    epa
))